In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("Hitters.csv")
df = df.copy()
df.head()
#veri setimizde index problemi yok 


,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [3]:
df.shape


(322, 20)

In [4]:
df.isnull().sum()
# salary değişkenlerimizde boş gözlemler bulunmaktadır 
# bunları ortalama ile dolduralım 


AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [5]:
df["Salary"].fillna(df.Salary.mean() , inplace = True)
# boş maaş değerlerini ortalama ile doldurduk aslında bunları ilerde tahmin ile dolduracağız

In [6]:
df.isnull().sum().any()
# datamızda boş değer var mı ? 


False

In [7]:
df.info()
# veri setimizdeki kategorik değişkenleri düzenliyelim 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 322 entries, 0 to 321
Data columns (total 20 columns):
AtBat        322 non-null int64
Hits         322 non-null int64
HmRun        322 non-null int64
Runs         322 non-null int64
RBI          322 non-null int64
Walks        322 non-null int64
Years        322 non-null int64
CAtBat       322 non-null int64
CHits        322 non-null int64
CHmRun       322 non-null int64
CRuns        322 non-null int64
CRBI         322 non-null int64
CWalks       322 non-null int64
League       322 non-null object
Division     322 non-null object
PutOuts      322 non-null int64
Assists      322 non-null int64
Errors       322 non-null int64
Salary       322 non-null float64
NewLeague    322 non-null object
dtypes: float64(1), int64(16), object(3)
memory usage: 50.4+ KB


# Kategorik değişkenleri modellemeye hazır hale getirme 

In [8]:
from sklearn.preprocessing import LabelEncoder 
cat_list = ["NewLeague" , "Division" , "League"] 

lbe = LabelEncoder()
for i in cat_list:
    df[i] = lbe.fit_transform(df[i])

In [9]:
df.head(3)
# tip dönüşümlerini başarılı bir şekilde uyguladık 

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,0,0,446,33,20,535.925882,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,1,1,632,43,10,475.000000,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,0,1,880,82,14,480.000000,0


# Bagimli ve bağımsız değişken ayrışımı 

In [10]:
X = df.drop("Salary"  , axis = 1) # bagımsız degisken 

In [11]:
y = df["Salary"] #bagimli değisken 

# Veri setini Train_Test olarak ayırma

In [12]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X , y , test_size = 0.20)

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
# doğru parcalamışız

(257, 19)
(257,)
(65, 19)
(65,)


# Model Kurma

In [20]:
from sklearn.linear_model import Lasso 
from sklearn.metrics import mean_squared_error , r2_score

In [21]:
las_model = Lasso(alpha = 0.03).fit(X_train , y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 12471742.945708029, tolerance: 4630.881545149098
  positive)


In [22]:
las_model.coef_

array([-2.49626703e+00,  7.74174540e+00,  4.24938336e+00, -2.11531717e+00,
       -5.76869440e-01,  7.05783887e+00, -1.56441177e+01, -7.44427442e-02,
        1.38194904e-02, -5.68533130e-01,  1.04466102e+00,  9.32560483e-01,
       -8.52901188e-01, -1.61499675e-01, -1.26534781e+02,  2.49811554e-01,
        3.32745422e-01, -3.83262344e+00, -2.37099960e+00])

## Tahmin , hata_kareleri

In [23]:
y_pred = las_model.predict(X_test)

In [24]:
np.sqrt( mean_squared_error(y_test , y_pred) )

273.8236466542002

In [25]:
r2_score(y_test , y_pred)

0.29596308871492116

## Cross val , optimum değer bulma

In [29]:
from sklearn.linear_model import LassoCV



In [32]:
las_cross = LassoCV(alphas = None , # program belirlesin
                    max_iter = 10000, # iterasyon sayısını belirtiyoruz
                    cv = 10 , #çaprazlama sayısı
                    normalize = True
                    
                    ).fit(X_train , y_train)

In [39]:
optimum_alp = las_cross.alpha_
# optimum alpha değeri
optimum_alp

0.15457852582617995

# final modelini kuralım 

In [40]:
from sklearn.linear_model import Lasso 
from sklearn.metrics import mean_squared_error ,r2_score

las_model = Lasso(alpha = optimum_alp,
                 normalize = True ).fit(X_train , y_train)


In [42]:
y_pred = las_model.predict(X_test)

array([ 970.22785498,  904.39562169,  230.14064197,  406.04521943,
        662.84578007,  531.16951285,  353.66410225, 1105.97893652,
        643.72915076,  293.7411623 ,  475.42479211,  495.85348935,
        537.24971912,  259.03490457,  478.60315094,  633.31327085,
        663.38936657, 1030.14378762,  559.51869628,  406.07679432,
        289.99884306,  334.66579503,  581.93313534,  453.0725116 ,
        566.26298616,  227.61570166, 1279.41519846,  331.77126659,
        524.35947055,  306.93088002,  457.84055591,  384.10639566,
        581.36014202,  581.59796485,  922.07133312,  747.81712296,
        328.72581987,  370.156189  ,  148.26018638,  638.36860853,
        337.63783635,  855.46639867,  774.52875956,  479.40602461,
        291.86741532,  430.63000316,  374.41857529,  365.06838628,
        196.66733604,  717.28890783,  207.1794339 ,  845.04713786,
        322.20912021,  389.66296382,  362.92298855,  387.07719375,
        311.25203489,  440.27615871, 1176.47545445,  577.12954

In [43]:
np.sqrt( mean_squared_error(y_test , y_pred) )

273.8236466542002

In [44]:
r2_score(y_test ,y_pred)

0.29596308871492116